In [3]:
import os
import io
import time
import json
import zipfile
import requests
from bs4 import BeautifulSoup as bs
import xlsxwriter

headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.149 Safari/537.36'}
no_incluir = ["Table", "SharesAxis", "Member", "LineItems", "Explanatory"]

def ListadoLinks():
    print("Descargado desde web lista de links... ")
    url = "https://www.bmv.com.mx/es/emisoras/archivos-estadar-xbrl"

    respuesta = requests.get(url, headers=headers, timeout=60)
    if "Sitio Grupo BMV no disponible" in respuesta.text:
        raise Exception("El servidor de la BMV se encuentra en mantenimiento. Intente más tarde.")

    sopadecoditos = bs(respuesta.content, "html.parser")
    tabla_zips = sopadecoditos.find("tbody").findAll("tr")
    d_links = {}

    print("[OK]")
    print("Preparando la información... ")

    for row in tabla_zips:
        texto_tabla = row.find_all("td")
        emisora = texto_tabla[0].get_text()
        link_tabla = row.find_all("a")
        link = str(link_tabla[0].get("href")).replace("/docs-pub/ifrsxbrl/../visor/visorXbrl.html?docins=../", "https://www.bmv.com.mx/docs-pub/")
        periodo = str(link_tabla[0].get_text()).replace("\n", "")
        lista_hija = []
        lista_padre = []

        if "anexon" not in link:
            lista_hija.append(link)
            lista_hija.append(periodo)
            lista_padre.append(lista_hija)

            if emisora in d_links:
                d_links[emisora].extend(lista_padre)
            else:
                d_links[emisora] = lista_padre
    return d_links
d_links = ListadoLinks()

print("[OK]")

def Questionario():
    while True:
        emisora_select = str(input("Escoger emisora:")).upper()
        try:
            print("_"*60)
            for i,item in enumerate(d_links[emisora_select]):
                print(f"{i+1}{'# '} {item[1]} {' #'}{i+1}")
            print("_"*60)
            print("\n")
            while True:
                try:
                    folio_select = input("¿Qué periodo?:")
                    try:
                        folio_select = int(folio_select)
                    except ValueError:
                        print("Selección erronea. Reintentar.")
                        continue
                    url = d_links[emisora_select][folio_select-1][0]
                    periodo_select = d_links[emisora_select][folio_select-1][1]
                    break
                except IndexError:
                    print(f"No tengo registros con el número '{folio_select}'. Reintentar.")
                    print("\n")
                    continue
            break
        except KeyError:
            print(f"No tengo registros con la emisora '{emisora_select}'. Reintentar.")
            print("\n")
            continue
    return emisora_select, periodo_select, url

emisora_select = Questionario()
periodo_select = emisora_select[1]
url = emisora_select[2]
emisora_select = emisora_select[0]

if periodo_select.find("para el") > 0:
    periodo_select = periodo_select.replace(periodo_select[periodo_select.find("para el"):len(periodo_select)], "")
periodo_select = periodo_select.replace("Descargar Información Del Trimestre ", "")
periodo_select = periodo_select.replace("Del Año ", "T_")
periodo_select = periodo_select.replace(" ", "")

print(f"Procesando {emisora_select} del periodo {periodo_select}")

def maquila_eeff(uuid_eeff, codigo_eeff, XBRL_Taxonomia, XBRL_HechosPorId, id_efectivo, uuid_inicio_efectivo, uuid_cierre_efectivo):
    eeff_del_periodo = {}
    eeff_del_periodo_ifrs = {}

    if uuid_eeff:
        for rol in XBRL_Taxonomia["RolesPresentacion"]:
            if codigo_eeff in rol["Nombre"]:
                catalogo_cuentas = str(rol["Estructuras"][0])
                break

        catalogo_cuentas = catalogo_cuentas.replace("[", "").replace("]", "").replace("{", "").replace("}", "").replace("'", "").replace(" ", "").split(",")

        for cuenta_ifrs in catalogo_cuentas:
            if "IdConcepto:" in cuenta_ifrs and not any(nope in cuenta_ifrs for nope in no_incluir):
                cuenta_ifrs = cuenta_ifrs.replace("IdConcepto:", "").replace("SubEstructuras:", "")
                eeff_del_periodo_ifrs[cuenta_ifrs] = ""
                cuenta = cuenta_ifrs.replace("ifrs-full_", "").replace("ifrs_mx-cor_20141205_", "")
                eeff_del_periodo[cuenta] = [XBRL_Taxonomia["ConceptosPorId"][cuenta_ifrs]["Etiquetas"]["es"]["http://www.xbrl.org/2003/role/label"]["Valor"]
                                                .replace(" [sinopsis]", "").replace("[sinopsis]", "")]

        for hecho, contenido in XBRL_HechosPorId.items():
            if contenido["IdContexto"] == uuid_eeff and contenido["IdConcepto"] in eeff_del_periodo_ifrs.keys():
                eeff_del_periodo[contenido["IdConcepto"].replace("ifrs-full_", "").replace("ifrs_mx-cor_20141205_", "")].append(contenido["ValorNumerico"])

        if codigo_eeff == "[520000]":
            eeff_del_periodo["CashAndCashEquivalents"] = [XBRL_Taxonomia["ConceptosPorId"][id_efectivo]["Etiquetas"]["es"]
                                                          ["http://www.xbrl.org/2003/role/periodStartLabel"]["Valor"]]
            eeff_del_periodo["CashAndCashEquivalents_Ending"] = [XBRL_Taxonomia["ConceptosPorId"][id_efectivo]["Etiquetas"]["es"]
                                                                 ["http://www.xbrl.org/2003/role/periodEndLabel"]["Valor"]]

            for key, value in XBRL_HechosPorId.items():
                if value["IdContexto"] == uuid_inicio_efectivo[2] and value["IdConcepto"] == id_efectivo:
                    eeff_del_periodo["CashAndCashEquivalents"] = [XBRL_Taxonomia["ConceptosPorId"][id_efectivo]["Etiquetas"]["es"]
                                                                  ["http://www.xbrl.org/2003/role/periodStartLabel"]["Valor"], value["ValorNumerico"]]

                if value["IdContexto"] == uuid_cierre_efectivo[2] and value["IdConcepto"] == id_efectivo:
                    eeff_del_periodo["CashAndCashEquivalents_Ending"] = [XBRL_Taxonomia["ConceptosPorId"][id_efectivo]["Etiquetas"]["es"]
                                                                         ["http://www.xbrl.org/2003/role/periodEndLabel"]["Valor"], value["ValorNumerico"]]

        return eeff_del_periodo


def tres_estados_financieros():
    l_contextos_unicos = []
    estados_financieros = {}
    uuids_maestros = {}

    while True:
        if str(url[-3:]).lower() == "zip":
            try:
                respuesta = requests.get(url, stream=True)
            except requests.exceptions.ConnectionError:
                print("El servidor rechaza solicitud, reintentando en 1 minuto.")
                time.sleep(60)
                continue
            archivo_zip = zipfile.ZipFile(io.BytesIO(respuesta.content), 'r')
            XBRL = json.loads(archivo_zip.read(archivo_zip.namelist()[0]))
        elif str(url[-4:]).lower() == "json":
            try:
                respuesta = requests.get(url)
            except requests.exceptions.ConnectionError:
                print("El servidor rechaza solicitud, reintentando en 1 minuto.")
                time.sleep(60)
                continue
            soupadecoditos = bs(respuesta.content, "html.parser")
            XBRL = json.loads(soupadecoditos.prettify())
        else:
            raise Exception(f"La URL '{url}' no termina en .zip o .json, verificar lista de links.")
        break

    id_periodo_reporte = XBRL["HechosPorIdConcepto"]["ifrs-full_DateOfEndOfReportingPeriod2013"][0]
    fecha_cierre_original = XBRL["HechosPorId"][id_periodo_reporte]["Valor"]
    fecha_cierre = fecha_cierre_original

    if int(fecha_cierre_original[0:4]) < 2010:
        id_moneda = "ifrs_DescriptionOfPresentationCurrency"
        id_nombre_emisora = "ifrs_NameOfReportingEntityOrOtherMeansOfIdentification"
    else:
        id_moneda = "ifrs-full_DescriptionOfPresentationCurrency"
        id_nombre_emisora = "ifrs-full_NameOfReportingEntityOrOtherMeansOfIdentification"

    moneda = str(XBRL["HechosPorId"][XBRL["HechosPorIdConcepto"][id_moneda][0]]["Valor"]).upper()
    emisora_select = str(XBRL["HechosPorId"][XBRL["HechosPorIdConcepto"][id_nombre_emisora][0]]["Valor"]).upper()

    for concepto in XBRL["ContextosPorId"]:
        if not XBRL["ContextosPorId"][concepto]["ContieneInformacionDimensional"]:
            l_contextos_unicos.append(XBRL["ContextosPorId"][concepto]["Id"])

    """Tranformacion de XBRL a JSON"""
    for ronda in range(2):
        if ronda == 1:
            fecha_cierre = str(int(fecha_cierre[0:4]) - 1) + fecha_cierre[4:10]

        anio_actual = str(fecha_cierre[0:4])
        fecha_inicio = anio_actual + "-01-01"
        fecha_inicio_y_cierre = fecha_inicio + "_" + fecha_cierre
        fecha_cierre_anterior = str(int(anio_actual) - 1) + "-12-31"
        fecha_cierre_actual = fecha_cierre

        if fecha_cierre[5:7] == "12":
            fecha_trimestre = f"{fecha_cierre[0:4]}-10-01_{fecha_cierre}"
        if fecha_cierre[5:7] == "09":
            fecha_trimestre = f"{fecha_cierre[0:4]}-07-01_{fecha_cierre}"
        if fecha_cierre[5:7] == "06":
            fecha_trimestre = f"{fecha_cierre[0:4]}-04-01_{fecha_cierre}"
        if fecha_cierre[5:7] == "03":
            fecha_trimestre = f"{fecha_cierre[0:4]}-01-01_{fecha_cierre}"

        if ronda == 1:
            fecha_cierre = str(fecha_cierre[0:4]) + "-12-31"

        if int(fecha_cierre[0:4]) < 2010:
            id_efectivo = "ifrs_CashAndCashEquivalents"
        else:
            id_efectivo = "ifrs-full_CashAndCashEquivalents"

        for unicontexto in l_contextos_unicos:
            try:
                if unicontexto in XBRL["ContextosPorFecha"][fecha_cierre]: uuids_maestros["uuid_posicion"] = ["posicion", "[210000]", unicontexto, fecha_cierre]
            except KeyError:
                uuids_maestros["uuid_posicion"] = ["posicion", "[210000]", False, fecha_cierre]
            try:
                if unicontexto in XBRL["ContextosPorFecha"][fecha_inicio_y_cierre]:
                    uuids_maestros["uuid_resultado"] = ["resultado_acumulado", "[310000]", unicontexto, fecha_inicio_y_cierre]
                    uuids_maestros["uuid_flujos"] = ["flujos", "[520000]", unicontexto, fecha_inicio_y_cierre]
            except KeyError:
                uuids_maestros["uuid_resultado"] = ["resultado_acumulado", "[310000]", False, fecha_inicio_y_cierre]
                uuids_maestros["uuid_flujos"] = ["flujos", "[520000]", False, fecha_inicio_y_cierre]
            try:
                if unicontexto in XBRL["ContextosPorFecha"][fecha_trimestre]: uuids_maestros["uuid_trimestre"] = ["resultado_trimestre", "[310000]", unicontexto, fecha_trimestre]
            except KeyError:
                uuids_maestros["uuid_trimestre"] = ["resultado_trimestre", "[310000]", False, False]
            try:
                if unicontexto in XBRL["ContextosPorFecha"][fecha_cierre_anterior]: uuids_maestros["uuid_inicio_efectivo"] = [False, False, unicontexto, False]
            except KeyError:
                uuids_maestros["uuid_inicio_efectivo"] = [False, False, False, False]
            try:
                if unicontexto in XBRL["ContextosPorFecha"][fecha_cierre_actual]: uuids_maestros["uuid_cierre_efectivo"] = [False, False, unicontexto, False]
            except KeyError:
                uuids_maestros["uuid_cierre_efectivo"] = [False, False, False, False]

        for nombre_uuid, llaves in uuids_maestros.items():
            if llaves[0] in ("posicion", "resultado_acumulado", "resultado_trimestre", "flujos") and llaves[2]:
                if llaves[0] not in estados_financieros.keys():
                    estados_financieros[llaves[0]] = {llaves[3]: maquila_eeff(llaves[2], llaves[1], XBRL["Taxonomia"], XBRL["HechosPorId"], id_efectivo, uuids_maestros["uuid_inicio_efectivo"], uuids_maestros["uuid_cierre_efectivo"])}
                else:
                    estados_financieros[llaves[0]] = {list(estados_financieros[llaves[0]].keys())[0]:
                                                          estados_financieros[llaves[0]][list(estados_financieros[llaves[0]].keys())[0]], llaves[3]:
                                                          maquila_eeff(llaves[2], llaves[1], XBRL["Taxonomia"], XBRL["HechosPorId"], id_efectivo, uuids_maestros["uuid_inicio_efectivo"], uuids_maestros["uuid_cierre_efectivo"])}

    """Inicio de maquila Excel"""
    try:
        os.mkdir(f"{os.path.abspath('')}/{emisora_select}")
    except FileExistsError:
        pass
    workbook = xlsxwriter.Workbook(f"{os.path.abspath('')}/{emisora_select}/{emisora_select}_{periodo_select}_EstadosFinancieros.xlsx")
    worksheet_balance = workbook.add_worksheet('Estado de Posicion Financiera')
    worksheet_resultados_ytd = workbook.add_worksheet('Estado de Resultados YTD')
    worksheet_resultados_trimestre = workbook.add_worksheet('Estado de Resultados Trimestre')
    worksheet_flujo = workbook.add_worksheet('Estado de Flujos de Efectivo')
    negritas_centrado = workbook.add_format({'bold':True})
    centrado = negritas_centrado.set_align('center')
    cashbaby = workbook.add_format({'num_format':'#,##0.00;[Red](#,##0.00)'})

    for ronda in range(2):

        if ronda == 0:
            columna_numeros = 2
        elif ronda == 1:
            columna_numeros = 3

        #Balance General
        try:
            worksheet_balance.write(0,0,emisora_select,negritas_centrado)
            worksheet_balance.write(0,2,f'Cifras en {moneda}',centrado)
            worksheet_balance.write(2,columna_numeros,str(list(estados_financieros["posicion"].keys())[ronda]),negritas_centrado)

            for i, (llave, valor) in enumerate(estados_financieros["posicion"][list(estados_financieros["posicion"].keys())[ronda]].items()):
                worksheet_balance.write(2+i,0,llave)
                worksheet_balance.write(2+i,1,valor[0])
                try:
                    worksheet_balance.write(2+i,columna_numeros,valor[1],cashbaby)
                except IndexError:
                    pass
        except (IndexError, KeyError):
            pass


        #Estado de Resultados YTD y Trimestre
        try:
            worksheet_resultados_ytd.write(0,0,emisora_select, negritas_centrado)
            worksheet_resultados_ytd.write(0,2,f'Cifras en {moneda}',centrado)
            worksheet_resultados_ytd.write(2,columna_numeros,str(list(estados_financieros["resultado_acumulado"].keys())[ronda]),negritas_centrado)

            for i, (llave, valor) in enumerate(estados_financieros["resultado_acumulado"][list(estados_financieros["resultado_acumulado"].keys())[ronda]].items()):
                worksheet_resultados_ytd.write(2+i,0,llave)
                worksheet_resultados_ytd.write(2+i,1,valor[0])
                try:
                    worksheet_resultados_ytd.write(2+i,columna_numeros,valor[1],cashbaby)
                except IndexError:
                    pass
        except (IndexError, KeyError):
            pass

        try:
            worksheet_resultados_trimestre.write(0,0,emisora_select, negritas_centrado)
            worksheet_resultados_trimestre.write(0,2,f'Cifras en {moneda}',centrado)
            worksheet_resultados_trimestre.write(2,columna_numeros,str(list(estados_financieros["resultado_trimestre"].keys())[ronda]),negritas_centrado)

            for i, (llave, valor) in enumerate(estados_financieros["resultado_trimestre"][list(estados_financieros["resultado_trimestre"].keys())[ronda]].items()):
                worksheet_resultados_trimestre.write(2+i,0,llave)
                worksheet_resultados_trimestre.write(2+i,1,valor[0])
                try:
                    worksheet_resultados_trimestre.write(2+i,columna_numeros,valor[1],cashbaby)
                except IndexError:
                    pass
        except (IndexError, KeyError):
            pass

        #Flujo de Efectivo
        try:
            worksheet_flujo.write(0,0,emisora_select, negritas_centrado)
            worksheet_flujo.write(0,2,f'Cifras en {moneda}',centrado)
            worksheet_flujo.write(2,columna_numeros,str(list(estados_financieros["flujos"].keys())[ronda]),negritas_centrado)

            for i, (llave, valor) in enumerate(estados_financieros["flujos"][list(estados_financieros["flujos"].keys())[ronda]].items()):
                worksheet_flujo.write(2+i,0,llave)
                worksheet_flujo.write(2+i,1,valor[0])
                try:
                    worksheet_flujo.write(2+i,columna_numeros,valor[1],cashbaby)
                except IndexError:
                    pass
        except (IndexError, KeyError):
            pass

    worksheet_balance.set_column('A:B',116)
    worksheet_balance.set_column('C:D',18)
    worksheet_resultados_ytd.set_column('A:B',68)
    worksheet_resultados_ytd.set_column('C:D',25)
    worksheet_resultados_trimestre.set_column('A:B',68)
    worksheet_resultados_trimestre.set_column('C:D',25)
    worksheet_flujo.set_column('A:B',108)
    worksheet_flujo.set_column('C:D',25)

    workbook.close()
    print("\n")
    print(f"EEFF Disponible en: {os.path.abspath('')}/{emisora_select}/{emisora_select}_{periodo_select}_EstadosFinancieros.xlsx")

tres_estados_financieros()

Descargado desde web lista de links... 
[OK]
Preparando la información... 
[OK]
____________________________________________________________
1#  Descargar Información Del Trimestre 2 Del Año 2021   #1
2#  Descargar Información Del Trimestre 1 Del Año 2021   #2
3#  Descargar Información Del Trimestre 4 Del Año 2020   #3
4#  Descargar Información Del Trimestre 3 Del Año 2020   #4
5#  Descargar Información Del Trimestre 2 Del Año 2020   #5
6#  Descargar Información Del Trimestre 1 Del Año 2020   #6
7#  Descargar Información Del Trimestre 4 Del Año 2019   #7
8#  Descargar Información Del Trimestre 3 Del Año 2019   #8
9#  Descargar Información Del Trimestre 2 Del Año 2019   #9
10#  Descargar Información Del Trimestre 1 Del Año 2019   #10
11#  Descargar Información Del Trimestre 4 Del Año 2018   #11
12#  Descargar Información Del Trimestre 3 Del Año 2018   #12
13#  Descargar Información Del Trimestre 2 Del Año 2018   #13
14#  Descargar Información Del Trimestre 1 Del Año 2018   #14
15#  Desc

dict_keys(['resultado_trimestre', 'resultado_acumulado', 'flujos', 'posicion'])